In [236]:
import os
import math as m
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import nbconvert

In [229]:
def angles(c1,c2,c3):
    d1 = 500
    a2 = 431.8
    d3 = 149.09
    a3 = -20.32
    d4 = 433.07
    d6 = 56.25
    base = np.matrix([[1,0,0,0], [0,1,0,0], [0,0,1,d1], [0,0,0,1,]])
    tool = np.matrix([[1,0,0,0], [0,1,0,0], [0,0,1,d6], [0,0,0,1,]])

    pos1 = np.matrix([[1, 0, 0, 460], [0, -1, 0, 390,], [0, 0, -1, 580,], [0, 0, 0, 1,]])
    pos2 = np.matrix([[0, 0, 1, 300], [1, 0, 0, -340,], [0, 1, 0, 220,], [0, 0, 0, 1]])
    pos3 = np.matrix([[0, 1, 0, 230], [0, 0, 1, 300], [1, 0, 0, 380], [0, 0, 0, 1]])
    
    t1 = base.I*pos1*tool.I
    t2 = base.I*pos2*tool.I
    t3 = base.I*pos3*tool.I
    t = [t1,t2,t3]
    theta1 = [0]*3
    theta2 = [0]*3
    theta3 = [0]*3
    theta4 = [0]*3
    theta5 = [0]*3
    theta6 = [0]*3
    state = [1,1,1]
#     theta = np.matrix([[0]*3,[1]*3,[2]*3])
    theta = [0]*18
    thetaLim = np.matrix([[-160,160],[-223,43],[-52,232],[-110,170],[-100,100],[-266,266]])
    thetaLim = thetaLim*m.pi/180
#     print(t1,'\n\n',t2,'\n\n',t3,'\n\n',thetaLim)
#     print(t)
#     print(thetaLim[0,1])
    ## using conditions to set parameters
    if c1 =='R':
        n = -1
    else:
        n = 1
    if c1 == 'L' and c2 == 'A':
        o = -1
    elif c1 == 'L' and c2 == 'B':
        o = 1
    elif c1 == 'R' and c2 == 'A':
        o = 1
    else:
        o = -1
    if c3 == 'NF':
        a = 1
        fp = 0
    else:
        a = -1
        fp = 1
    for i in range(3):
        
        theta[i] = m.atan2(t[i][1,3],t[i][0,3])-m.atan2(d3,(n*m.sqrt(t[i][0,3]**2+t[i][1,3]**2-d3**2)))
        d = (m.cos(theta[i])*t[i][0,3]+m.sin(theta[i])*t[i][1,3])**2+t[i][2,3]**2-d4**2-a3**2-a2**2
        e = 4*a2**2*(a3**2+d4**2)
        theta[i+6] = m.atan2(d,(o*m.sqrt(e-d**2)))-m.atan2(a3,d4)
        ks2 =  (d4*m.sin(theta[i+6])+a3*m.cos(theta[i+6])+a2)  *(-t[i][2,3])  +(d4*m.cos(theta[i+6])-a3*m.sin(theta[i+6]))   * (m.cos(theta[i])*t[i][0,3]+m.sin(theta[i])*t[i][1,3])
        kc2 = -(d4*m.cos(theta[i+6])-a3*m.sin(theta[i+6]))*      (-t[i][2,3])  +(d4*m.sin(theta[i+6])+a3*m.cos(theta[i+6])+a2)* (m.cos(theta[i])*t[i][0,3]+m.sin(theta[i])*t[i][1,3])
        
        theta[i+3] = m.atan2(ks2,kc2)
                
    
        if theta[i] < thetaLim[0,0]:
            theta[i] = theta[i] + 2*m.pi
        elif theta[i] > thetaLim[0,1]:
            theta[i] = theta[i] - 2*m.pi
        if theta[i] < thetaLim[0,0]:
            theta[i] = thetaLim[0,0]
            state[i] = 0
        elif theta[i] > thetaLim[0,1]:
            theta[i] = thetaLim[0,0]
            state[i] = 0

        if theta[i+6] < thetaLim[2,0]:
            theta[i+6] = theta[i+6] + 2*m.pi
        elif theta[i+6] > thetaLim[2,1]:
            theta[i+6] = theta[i+6] - 2*m.pi
        if theta[i+6] < thetaLim[2,0]:
            theta[i+6] = thetaLim[2,1]
            state[i] = 0
        elif theta[i+6] > thetaLim[2,1]:
            theta[i+6] = thetaLim[2,0]
            state[i] = 0

        if theta[i+3] < thetaLim[1,0]:
            theta[i+3] = theta[i+3] + 2*m.pi
        elif theta[i+3] > thetaLim[1,1]:
            theta[i+3] = theta[i+3] - 2*m.pi
        if theta[i+3] < thetaLim[1,0]:
            theta[i+3] = thetaLim[1,1]
            state[i] = 0
        elif theta[i+3] > thetaLim[1,1]:
            theta[i+3] = thetaLim[1,0]
            state[i] = 0

        theta23 = theta[i+3] + theta[i+6]
        s5Temp = ((m.cos(theta[i])*t[i][0,2]+m.sin(theta[i])*t[i][1,2])*m.cos(theta23)-t[i][2,2]*m.sin(theta23))**2+(-m.sin(theta[i])*t[i][0,2]+m.cos(theta[i])*t[i][1,2])**2
    #     s5Temp = ((m.cos(theta[i])*t[i][0,2] + sin(theta[i])*t[i][1,2])*cos(theta23)  -t[i][2,2] *sin(theta23))**2 + (-sin(theta[i])* t[i][0,2]  + cos(theta[i]*t[i][1,2])**2            
        
#         print(s5Temp)
        
        if s5Temp==0 and i==1:
            theta[i+12] = 0
            theta46 = m.atan2((-m.sin(theta[i])*t[i][0,0]+m.cos(theta[i])*t[i][1,0]), (-m.sin(theta[i])*t[i][0,1]+m.cos(theta[i])*t[i][1,1]))
            theta[i+9] = 0
            theta[i+15] = theta46
            if theta[i+15] < thetaLim[5,0]:
                theta[i+15] = theta[i+15] + 2*m.pi
            elif theta[i+15] > thetaLim[5,1]:
                theta[i+15] = theta[i+15] - 2*m.pi
            if theta[i+15] < thetaLim[5,0]:
                theta[i+15] = thetaLim[5,1]
                state[i] = 0
            elif theta[i+15] > thetaLim[5,1]:
                theta[i+15] = thetaLim[5,0]
                state[i] = 0
        elif s5Temp==0 and i>1:
            theta[i+12] = 0
            theta46 = m.atan2((-m.sin(theta[i])*t[i][0,0]+m.cos(theta[i])*t[i][1,0]), (-m.sin(theta[i])*t[i][0,1]+m.cos(theta[i])*t[i][1,1]))
            theta[i+9] = theta[i+8]
            if theta[i+9] < thetaLim[3,0]:
                theta[i+9] = theta[i+9] + 2*m.pi
            elif theta[i+9] > thetaLim[3,1]:
                theta[i+9] = theta[i+9] - 2*m.pi
            if theta[i+9] < thetaLim[3,0]:
                theta[i+9] = thetaLim[3,1]
                state[i] = 0
            elif theta[i+9] > thetaLim[3,1]:
                theta[i+9] = thetaLim[3,0]
                state[i] = 0
            theta[i+15] = theta46 - theta[i+9]
            if theta[i+15] < thetaLim[5,0]:
                theta[i+15] = theta[i+15] + 2*m.pi
            elif theta[i+15] > thetaLim[5,1]:
                theta[i+15] = theta[i+15] - 2*m.pi
            if theta[i+15] < thetaLim[5,0]:
                theta[i+15] = thetaLim[5,1]
                state[i] = 0
            elif theta[i+15] > thetaLim[5,1]:
                theta[i+15] = thetaLim[5,0]
                state[i] = 0
        else:
            theta[i+12] = m.atan2(a*m.sqrt(s5Temp),((m.cos(theta[i])*t[i][0,2]+m.sin(theta[i])*t[i][1,2])*m.sin(theta23)+t[i][2,2]*m.cos(theta23)))
            if theta[i+12] < thetaLim[4,0]:
                theta[i+12] = theta[i+12] + 2*m.pi
            elif theta[i+12] > thetaLim[4,1]:
                theta[i+12] = theta[i+12] - 2*fp*m.pi
            if theta[i+12] < thetaLim[4,0]:
                theta[i+12] = thetaLim[4,1]
                state[i] = 0
            elif theta[i+12] > thetaLim[4,1]:
                theta[i+12] = thetaLim[4,0]
                state[i] = 0
            theta[i+9] = m.atan2((-m.sin(theta[i])*t[i][0,2]+m.cos(theta[i])*t[i][1,2]),((m.cos(theta[i])*t[i][0,2]+m.sin(theta[i])*t[i][1,2])*m.cos(theta23)-t[i][2,2]*m.sin(theta23)))+fp*m.pi
            if theta[i+9] < thetaLim[3,0]:
                theta[i+9] = theta[i+9] + 2*m.pi
            elif theta[i+9] > thetaLim[3,1]:
                theta[i+9] = theta[i+9] - 2*m.pi
            if theta[i+9] < thetaLim[3,0]:
                theta[i+9] = thetaLim[3,1]
                state[i] = 0
            elif theta[i+9] > thetaLim[3,1]:
                theta[i+9] = thetaLim[3,0]
                state[i] = 0
            theta[i+15] = m.atan2(((m.cos(theta[i])*t[i][0,1]+m.sin(theta[i])*t[i][1,1])*m.sin(theta23)+t[i][2,1]*m.cos(theta23)),-((m.cos(theta[i])*t[i][0,0]+m.sin(theta[i])*t[i][1,0])*m.sin(theta23)+t[i][2,0]*m.cos(theta23)))+fp*m.pi
            if theta[i+15] < thetaLim[5,0]:
                theta[i+15] = theta[i+15] + 2*m.pi
            elif theta[i+15] > thetaLim[5,1]:
                theta[i+15] = theta[i+15] - 2*m.pi
            if theta[i+15] < thetaLim[5,0]:
                theta[i+15] = thetaLim[5,1]
                state[i] = 0
            elif theta[i+15] > thetaLim[5,1]:
                theta[i+15] = thetaLim[5,0]
                state[i] = 0
    print('__'*42)
    print('\n','\t'*4,c1,'\t',c2,'\t',c3,'\n'*2)
    if 0 in state:
        print('-Not Feasible-\n')
    else:
        print('-Feasible-\n')
    for j in range(3):
        theta1[j] = theta[j]
        theta2[j] = theta[j+3]
        theta3[j] = theta[j+6]
        theta4[j] = theta[j+9]
        theta5[j] = theta[j+12]
        theta6[j] = theta[j+15]
    print('\t'*2,'Pos1','\t'*3,'Pos2','\t'*3,'Pos3','\n')
    print('Theta 1','\t',theta1[0]*180/m.pi,'\t',theta1[1]*180/m.pi,'\t',theta1[2]*180/m.pi,'\n')
    print('Theta 2','\t',theta2[0]*180/m.pi,'\t',theta2[1]*180/m.pi,'\t',theta2[2]*180/m.pi,'\n')
    print('Theta 3','\t',theta3[0]*180/m.pi,'\t',theta3[1]*180/m.pi,'\t',theta3[2]*180/m.pi,'\n')
    print('Theta 4','\t'*2,theta4[0]*180/m.pi,'\t'*2,theta4[1]*180/m.pi,'\t',theta4[2]*180/m.pi,'\n')
    print('Theta 5','\t',theta5[0]*180/m.pi,'\t',theta5[1]*180/m.pi,'\t',theta5[2]*180/m.pi,'\n')
    print('Theta 6','\t',theta6[0]*180/m.pi,'\t',theta6[1]*180/m.pi,'\t',theta6[2]*180/m.pi,'\n')
    
    
    for ii in range(3):
        poses = ['Pos1','Pos2','Pos3']
        t1 = theta1[ii]
        t2 = theta2[ii]
        t3 = theta3[ii]
        t4 = theta4[ii]
        t5 = theta5[ii]
        t6 = theta6[ii]
     
        A1 = np.matrix([[m.cos(t1),0,-m.sin(t1),0],[m.sin(t1),0,m.cos(t1),0],[0,-1,0,0],[0,0,0,1]])
        A2 = np.matrix([[m.cos(t2),-m.sin(t2),0,a2*m.cos(t2)],[m.sin(t2),m.cos(t2),0,a2*m.sin(t2)],[0,0,1,0],[0,0,0,1]])
        A3 = np.matrix([[m.cos(t3),0,m.sin(t3),a3*m.cos(t3)],[m.sin(t3),0,-m.cos(t3),a3*m.sin(t3)],[0,1,0,d3],[0,0,0,1]])
        A4 = np.matrix([[m.cos(t4),0,-m.sin(t4),0],[m.sin(t4),0,m.cos(t4),0],[0,-1,0,d4],[0,0,0,1]])
        A5 = np.matrix([[m.cos(t5),0,m.sin(t5),0],[m.sin(t5),0,-m.cos(t5),0],[0,1,0,0],[0,0,0,1]])
        A6 = np.matrix([[m.cos(t6),-m.sin(t6),0,0],[m.sin(t6),m.cos(t6),0,0],[0,0,1,0],[0,0,0,1]]) 
        
        tran = A1*A2*A3*A4*A5*A6
        pos = base*tran*tool
        
        print('\n',poses[ii],'\n')
        tt = pos.round(decimals=5)
        print(tt)

    print('__'*42)
    

In [235]:
angles('R','A','NF')
angles('L','A','NF')
angles('R','B','NF') 
angles('L','B','NF')
angles('R','A','F')
angles('L','A','F')
angles('R','B','F')
angles('L','B','F')

____________________________________________________________________________________

 				 R 	 A 	 NF 


-Not Feasible-

		 Pos1 			 Pos2 			 Pos3 

Theta 1 	 -125.39501216646929 	 146.51516470186624 	 -106.92261646675178 

Theta 2 	 -120.65473049617394 	 -159.19671347940758 	 -133.4387358715931 

Theta 3 	 0.485671040194677 	 -19.804786072730202 	 -43.445719841476404 

Theta 4 		 170.0 		 -33.48883914275183 	 -16.946231223109365 

Theta 5 	 59.830940544020756 	 89.16723163015169 	 87.01948722974295 

Theta 6 	 -125.39501216646929 	 -89.44907227309349 	 0.9076911107842234 


 Pos1 

[[ 9.8862000e-01  8.1910000e-02  1.2620000e-01  4.6709894e+02]
 [ 9.2630000e-02 -9.9235000e-01 -8.1570000e-02  3.8541145e+02]
 [ 1.1856000e-01  9.2340000e-02 -9.8864000e-01  5.8063874e+02]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]

 Pos2 

[[  -0.   -0.    1.  300.]
 [   1.    0.    0. -340.]
 [  -0.    1.    0.  220.]
 [   0.    0.    0.    1.]]

 Pos3 

[[ -0.   1.   0. 230.]
 [ -0.